In [ ]:
from thetaray.api.context import init_context
from thetaray.api.dataset import dataset_functions
from thetaray.common.data_environment import DataEnvironment
from thetaray.api.graph import publish_nodes, publish_edges
from thetaray.api.evaluation import read_alerted_activities, load_evaluated_activities
import random
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pandas as pd
from thetaray.common import Constants
from functools import reduce
from pyspark.sql import DataFrame

In [ ]:
from common.libs.config.loader import load_config
from thetaray.api.context import init_context
import datetime
from thetaray.common import Constants

from common.libs.config.loader import load_config
from common.libs.config.basic_execution_config_loader import BasicExecutionConfig, DevBasicExecutionConfig
from common.libs.context_utils import is_run_triggered_from_airflow

import datetime
import dateutil.relativedelta
import json
import logging
from pyspark.sql import DataFrame, Window, functions as f
from pyspark.sql import SQLContext
from pyspark.sql.types import LongType
from pyspark.sql.functions import col, when,count
from common.libs import dates as dates_lib
from common.libs import features_discovery
from thetaray.common.data_environment import DataEnvironment
from common.libs.features_executor import FeaturesExecutor
from common.libs.feature_engineering import max_look_back_monthly_features, max_look_back_daily_weekly_features
from common.libs.zscore import enrich_with_z_score
from common.factory.wrangling_execution_strategy import get_wrangling_execution_strategy
from common.factory.eval_flow_definition import get_evaluation_flow_definition
from common.factory.domain_definition import get_domain_definition
from common.notebook_utils.wrangling.wrangling_execution_strategy import WranglingExecutionStrategy
from common.definitions.domain import DomainDefinition
from common.definitions.eval_flow import EvaluationFlowDefinition
from common.libs.context_utils import get_dataset

from thetaray.api.context import init_context
from thetaray.api.dataset import dataset_functions
from thetaray.api.solution import IngestionMode
from thetaray.common import Constants
from thetaray.common.data_environment import DataEnvironment

logging.getLogger().handlers[0].setFormatter(logging.Formatter(fmt='%(levelname)s: %(asctime)s @ %(message)s',datefmt='%Y-%m-%d %H:%M:%S'))
logging.basicConfig(level=logging.INFO)

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from common.libs.config.loader import load_config
from thetaray.api.context import init_context
import datetime
from thetaray.common import Constants

from common.libs.config.loader import load_config
from common.libs.config.basic_execution_config_loader import BasicExecutionConfig, DevBasicExecutionConfig
from common.libs.context_utils import is_run_triggered_from_airflow

execution_date = Constants.BEGINNING_OF_TIME

if is_run_triggered_from_airflow():
    context = init_context()
    basic_execution_config = BasicExecutionConfig(domain=context.domain,
                                                  stage=context.parameters['stage'],
                                                  entity=context.parameters['entity'],
                                                  cadence=context.parameters['cadence'],
                                                  spark_conf=context.spark_conf)
else:
    basic_execution_config = DevBasicExecutionConfig()
    context = init_context(execution_date=execution_date,
                           domain=basic_execution_config.domain,
                           spark_conf=basic_execution_config.spark_conf)


print(basic_execution_config)
spark = context.get_spark_session()
sc = SQLContext(spark)
params = context.parameters
print(f"Spark UI URL: {context.get_spark_ui_url()}")
print(json.dumps(params, indent=4))

domain = basic_execution_config.domain
entity = basic_execution_config.entity
cadence = basic_execution_config.cadence

In [ ]:
from common.libs.network_visualization import create_alert_graph_entities
from common.factory.eval_flow_definition import get_evaluation_flow_definition, EvaluationFlowDefinition

eval_flow_def: EvaluationFlowDefinition = get_evaluation_flow_definition(basic_execution_config)

evaluation_flow_identifier = eval_flow_def.eval_flow_id

alerted_activities_df = read_alerted_activities(context, evaluation_flow_identifier,data_environment=DataEnvironment.PUBLIC)
evaluated_activities_df = load_evaluated_activities(context, evaluation_flow_identifier,data_environment=DataEnvironment.PUBLIC)
alert_nodes_df, alert_edges_df = create_alert_graph_entities(alerted_activities_df=alerted_activities_df,
                                                             evaluated_activities_df=evaluated_activities_df,
                                                             entity_id_column_name=eval_flow_def.investigated_entity_id_column_name,
                                                             date_column_name=eval_flow_def.date_column_name)
alert_nodes_df=alert_nodes_df.withColumnRenamed('activity_id', 'AI')
alert_nodes_df=alert_nodes_df.withColumnRenamed('risk_id', 'RI')
alert_nodes_df=alert_nodes_df.withColumnRenamed('suppressed', 'SP')
publish_nodes(context=context, nodes_df=alert_nodes_df, graph_identifier='graph', node_type='AL',data_environment=DataEnvironment.PUBLIC)
publish_edges(context=context, edges_df=alert_edges_df, graph_identifier='graph', edge_type='AL', 
              source_node_type='AL', target_node_type='AC', data_environment=DataEnvironment.PUBLIC)

In [ ]:
context.close()